In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model

# Load your cleaned dataset (replace 'cleaned_dataset.csv' with the actual file name)
df_cleaned = pd.read_csv('cleaned_dataset.csv')

# Assuming 'target' is the column you want to predict
target_column = 'Prediction'

# Split the dataset into features (X) and target variable (y)
X = df_cleaned.drop(columns=[target_column])
y = df_cleaned[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Neural Network model
model = Sequential()

# Add input layer and first hidden layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))

# Add output layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training set
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


Epoch 1/10
104/104 [==============================] - 1s 5ms/step - loss: 0.3568 - accuracy: 0.8510 - val_loss: 0.1864 - val_accuracy: 0.9348
Epoch 2/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0971 - accuracy: 0.9686 - val_loss: 0.1442 - val_accuracy: 0.9589
Epoch 3/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0628 - accuracy: 0.9831 - val_loss: 0.1234 - val_accuracy: 0.9650
Epoch 4/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0471 - accuracy: 0.9849 - val_loss: 0.1140 - val_accuracy: 0.9686
Epoch 5/10
104/104 [==============================] - 1s 7ms/step - loss: 0.0339 - accuracy: 0.9930 - val_loss: 0.1116 - val_accuracy: 0.9662
Epoch 6/10
104/104 [==============================] - 1s 6ms/step - loss: 0.0283 - accuracy: 0.9943 - val_loss: 0.1070 - val_accuracy: 0.9638
Epoch 7/10
104/104 [==============================] - 1s 6ms/step - loss: 0.0224 - accuracy: 0.9961 - val_loss: 0.1281 - val_accuracy: 0.9457
Epoch 

In [1]:
!pip install keras==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.14.0 requires keras<2.15,>=2.14.0, but you have keras 2.12.0 which is incompatible.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Load your cleaned dataset (replace 'cleaned_dataset.csv' with the actual file name)
df_cleaned = pd.read_csv('cleaned_dataset.csv')

# Assuming 'target' is the column you want to predict
target_column = 'Prediction'

# Split the dataset into features (X) and target variable (y)
X = df_cleaned.drop(columns=[target_column])
y = df_cleaned[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the base model for KerasClassifier
def create_model(learning_rate=0.001, units=64):
    model = Sequential()
    model.add(Dense(units=units, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(units=1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Wrap the Keras model in a scikit-learn classifier
keras_model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define the hyperparameter grid to search
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'units': [32, 64, 128],
}

# Create a GridSearchCV object
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3, scoring='accuracy')

# Perform the grid search
grid_result = grid.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_result.best_params_)

# Make predictions on the test set using the best model
y_pred_prob = grid_result.best_estimator_.predict(X_test)
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Evaluate the performance of the best model
result_accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {result_accuracy:.2f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


<ipython-input-5-80de1badea61>:41: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)


44/44 [==============================] - 0s 2ms/step
Best Hyperparameters: {'learning_rate': 0.001, 'units': 32}
33/33 [==============================] - 0s 2ms/step
Accuracy: 0.97
Confusion Matrix:
 [[717  22]
 [  6 290]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       739
           1       0.93      0.98      0.95       296

    accuracy                           0.97      1035
   macro avg       0.96      0.97      0.97      1035
weighted avg       0.97      0.97      0.97      1035

